<a href="https://colab.research.google.com/github/haidour18/Benchemarking-Recommender-systems-/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install surprise
!pip install apyori
!pip install matrix_factorization
!pip install factor-analyzer
!pip install python-helper

In [57]:
#All the header files required for the code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import helper
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob , csv
import time, joblib
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
from zipfile import ZipFile as zip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [58]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

Movies


In [ ]:
!kaggle datasets download -d rounakbanik/the-movies-dataset


 95% 216M/228M [00:01<00:00, 120MB/s]
100% 228M/228M [00:02<00:00, 115MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [59]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = '/content/the-movies-dataset.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [60]:
#Preprocessing 
Movies_dataset =pd.read_csv("movies_metadata.csv")
Movies_dataset.head(10)
movies_overview= Movies_dataset[['id','genres','original_title' ,'overview']]

movies_overview.head()

movies_overview.columns
movies_overview.isnull().sum()
movies_overview.dropna(subset=['overview'], inplace=True)

In [61]:
#Remplacer les valeurs nuls

movies_overview.isnull().sum()
movies_overview.count
movies_overview.drop(movies_overview.tail(20000).index,inplace=True) # drop last n rows
movies_overview.count

<bound method DataFrame.count of            id  ...                                           overview
0         862  ...  Led by Woody, Andy's toys live happily in his ...
1        8844  ...  When siblings Judy and Peter discover an encha...
2       15602  ...  A family wedding reignites the ancient feud be...
3       31357  ...  Cheated on, mistreated and stepped on, the wom...
4       11862  ...  Just when George Banks has recovered from his ...
...       ...  ...                                                ...
24716  255476  ...  Cowgirls 'n Angels Dakota's Summer tells the s...
24717  253899  ...  Rick Morgan, an American engineer who runs a m...
24718   38470  ...  A treasury agent on the trail of counterfeit m...
24719  125257  ...  Two boys, whose parents ply their trade by the...
24720  207933  ...  A Harvard Medical School graduate takes a posi...

[24512 rows x 4 columns]>

In [62]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(movies_overview['overview'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim
overviews = movies_overview['overview']
movies_titles = movies_overview['original_title']
movies_genre = movies_overview['genres']
indices = pd.Series(movies_overview.index, index = movies_overview['overview'])

In [65]:
#Function that gets book recommendations based on the cosine similarity score of book titles
def movies_recommendations(overview, n):
    idx = indices[overview]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:n+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies_titles[movie_indices],movies_genre[movie_indices],overviews[movie_indices]

In [66]:
#Recommend n books for a book having index 1
movie_index = 1
n = 2

print([movie_index],movies_overview['original_title'][movie_index])
result =movies_recommendations(movies_overview.overview[movie_index], n)
result

[1] Jumanji


(9475          The Ring Two
 21473    Somewhere Between
 Name: original_title, dtype: object,
 9475     [{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...
 21473                  [{'id': 99, 'name': 'Documentary'}]
 Name: genres, dtype: object,
 9475     Rachel Keller must prevent evil Samara from ta...
 21473    Questions of race, identity and heritage are e...
 Name: overview, dtype: object)

TF-IDF on medium


**New Section**


In [67]:
#TF_IDF on medium

#load the dataset 
!kaggle datasets download -d hsankesara/medium-articles

medium-articles.zip: Skipping, found more recently modified local copy (use --force to force download)


In [68]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = '/content/medium-articles.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [69]:
articles = pd.read_csv("articles.csv")

articles.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


In [70]:
#Preprocessing 

articles_text =articles[['title', 'text']]

articles_text.head()

articles_text.columns
articles_text.isnull().sum()

title    0
text     0
dtype: int64

In [71]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(articles_text['text'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim
texts = articles_text['text']
title =  articles_text['title']
indices = pd.Series(articles_text.index, index =  articles_text['text'])
#Function that gets articles recommendations based on the cosine similarity score of texts
def articles_recommendations(text, n):
    idx = indices[text]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:n+1]
    text_indices = [i[0] for i in sim_scores]
    return title[text_indices]

In [72]:
#Recommend n movies for a book having index 1
article_index = 1
n = 6
print([article_index],articles['title'][article_index])
result =articles_recommendations(articles.text[article_index], n)
result

[1] Python for Data Science: 8 Concepts You May Have Forgotten


7     The Big List of DS/ML Interview Resources – To...
91    The Big List of DS/ML Interview Resources – To...
2     Automated Feature Engineering in Python – Towa...
44    Machine Learning Exercises In Python, Part 1 –...
13     Machine Learning is Fun! – Adam Geitgey – Medium
36     Machine Learning is Fun! – Adam Geitgey – Medium
Name: title, dtype: object

Goodbooks


```
Goodbooks```



In [94]:
#TF_IDF on goodbooks

#load the dataset 
!kaggle datasets download -d zygmunt/goodbooks-10k

goodbooks-10k.zip: Skipping, found more recently modified local copy (use --force to force download)


In [95]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = 'goodbooks-10k.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [118]:
books=pd.read_csv("books.csv")
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [119]:
#Preprocessing 
books_title.head()
books_title.columns
books_title.isnull().sum()
books_title.head()

,title
0,"The Hunger Games (The Hunger Games, #1)"
1,Harry Potter and the Sorcerer's Stone (Harry P...
2,"Twilight (Twilight, #1)"
3,To Kill a Mockingbird
4,The Great Gatsby


In [124]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(books['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim
title = books['title']
original=books['original_title']
indices = pd.Series(books_title.index, index = books['title'])
#Function that gets articles recommendations based on the cosine similarity score of texts
def book_recommendations(title, n):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:n+1]
    text_indices = [i[0] for i in sim_scores]
    return original[text_indices]

In [126]:
#Recommend n movies for a book having index 1
book_index = 4
n = 6
print([book_index],books.title[book_index])
result =book_recommendations(books.original_title[book_index], n)
result

[4] The Great Gatsby


7485                 Great House
159           Great Expectations
8869             The Great Brain
1105           The Great Divorce
3962           The Great Santini
5803    Sometimes a Great Notion
Name: original_title, dtype: object